In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('co-emissions-per-capita.csv')
top_n_row = 10
df

,Entity,Code,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,AFG,1949,0.001992
1,Afghanistan,AFG,1950,0.011266
2,Afghanistan,AFG,1951,0.012098
3,Afghanistan,AFG,1952,0.011946
4,Afghanistan,AFG,1953,0.013685
...,...,...,...,...
26595,Zimbabwe,ZWE,2018,0.711830
26596,Zimbabwe,ZWE,2019,0.636645
26597,Zimbabwe,ZWE,2020,0.500945
26598,Zimbabwe,ZWE,2021,0.524972


## 1. First Section

### 1.1 Top N countries with the most CO2 emissions chosen 1 year 

In [3]:
df_last_year = df[df['Year']==max(df['Year'])]
df_last_year = df_last_year[~df_last_year['Code'].isna()]
df_last_year = df_last_year.sort_values('Annual CO₂ emissions (per capita)', ascending=False).reset_index(drop=True)
df_last_year.iloc[:top_n_row].to_csv('co-emissions-per-capita-last-year.csv')
# df_last_year
px.bar(df_last_year.iloc[:top_n_row], x='Entity', y='Annual CO₂ emissions (per capita)')

### 1.1 Top N countries with the most CO2 emissions chosen a decade 

In [4]:
df_last_decade = df[df['Year']>max(df['Year'])-10]
df_last_decade = df_last_decade[~df_last_decade['Code'].isna()]
df_last_decade = df_last_decade[['Entity', 'Code', 'Annual CO₂ emissions (per capita)']].groupby(['Entity', 'Code']).mean().reset_index()
df_last_decade = df_last_decade.sort_values('Annual CO₂ emissions (per capita)', ascending=False).reset_index(drop=True)
df_last_decade.iloc[:top_n_row].to_csv('co-emissions-per-capita-last-decade.csv')
# df_last_decade
px.bar(df_last_decade.iloc[:top_n_row], x='Entity', y='Annual CO₂ emissions (per capita)')

## 2. Second Section

### 2.1 Regions comparison of CO2 emissions (chosen 1 year)

In [5]:
df_last_year = df[df['Year']==max(df['Year'])]
df_last_year = df_last_year[~df_last_year['Code'].isna()].reset_index(drop=True)
df_last_year

,Entity,Code,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,AFG,2022,0.295364
1,Albania,ALB,2022,1.743200
2,Algeria,DZA,2022,3.927226
3,Andorra,AND,2022,4.617124
4,Angola,AGO,2022,0.451552
...,...,...,...,...
210,Wallis and Futuna,WLF,2022,2.281908
211,World,OWID_WRL,2022,4.658219
212,Yemen,YEM,2022,0.337017
213,Zambia,ZMB,2022,0.445701


In [6]:
df_country_region_mapping = pd.read_csv('country-region-mapping.csv')
df_country_region_mapping

,Entity,Region
0,Afghanistan,Asia
1,Albania,Europe
2,Algeria,Africa
3,Andorra,Europe
4,Angola,Africa
...,...,...
210,Wallis and Futuna,Oceania
211,World,Global
212,Yemen,Asia
213,Zambia,Africa


In [7]:
df_last_year_with_regions = pd.merge(df_last_year, df_country_region_mapping, on='Entity', how='left')
df_last_year_with_regions

,Entity,Code,Year,Annual CO₂ emissions (per capita),Region
0,Afghanistan,AFG,2022,0.295364,Asia
1,Albania,ALB,2022,1.743200,Europe
2,Algeria,DZA,2022,3.927226,Africa
3,Andorra,AND,2022,4.617124,Europe
4,Angola,AGO,2022,0.451552,Africa
...,...,...,...,...,...
210,Wallis and Futuna,WLF,2022,2.281908,Oceania
211,World,OWID_WRL,2022,4.658219,Global
212,Yemen,YEM,2022,0.337017,Asia
213,Zambia,ZMB,2022,0.445701,Africa


In [19]:
top_n = 5

df_last_year_with_regions_sorted = df_last_year_with_regions.sort_values(by=['Region', 'Annual CO₂ emissions (per capita)'], ascending=[True, False])

df_last_year_with_regions_sorted['Rank'] = df_last_year_with_regions_sorted.groupby('Region')['Annual CO₂ emissions (per capita)'].rank(method='first', ascending=False)

df_last_year_with_regions_sorted['Entity'] = df_last_year_with_regions_sorted.apply(lambda row: row['Entity'] if row['Rank'] <= top_n else 'Other', axis=1)

df_last_year_with_regions_aggregated = df_last_year_with_regions_sorted.groupby(['Region', 'Entity'], as_index=False).agg({'Annual CO₂ emissions (per capita)': 'sum', 'Rank' : 'min'}).reset_index()

df_last_year_with_regions_aggregated = df_last_year_with_regions_aggregated.sort_values(by=['Region', 'Rank'], ascending=True)

df_last_year_with_regions_aggregated

,index,Region,Entity,Annual CO₂ emissions (per capita),Rank
1,1,Africa,Libya,9.242238,1.0
5,5,Africa,South Africa,6.746164,2.0
4,4,Africa,Seychelles,6.149512,3.0
0,0,Africa,Algeria,3.927226,4.0
3,3,Africa,Saint Helena,3.298648,5.0
2,2,Africa,Other,36.672683,6.0
10,10,Asia,Qatar,37.601273,1.0
11,11,Asia,United Arab Emirates,25.833244,2.0
6,6,Asia,Bahrain,25.672274,3.0
8,8,Asia,Kuwait,25.578102,4.0
